In [ ]:
from telethon import TelegramClient
import asyncio
import json
import os
import pandas as pd
import re
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env

api_id = os.getenv("API_ID")
api_hash = os.getenv("API_HASH")
session_name = 'ethio_session'
phone = '+251954504193'  # remove spaces


# === List of e-commerce Telegram channels ===
channels = [
    'Leyueqa',
    'nevacomputer',
    'ZemenExpress',
    'meneshayeofficial',
    'ethio_brand_collection'
]

# === Directory setup ===
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/clean', exist_ok=True)

# === Amharic text preprocessing function ===
def preprocess_amharic(text):
    if not text:
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    text = text.replace("\u1369", "1")  # Ethiopic digit example
    return text

# === Fetch messages function ===
async def fetch_messages(channel):
    channel_entity = await client.get_entity(channel)
    messages = []
    async for message in client.iter_messages(channel_entity, limit=100):
        all_records = []
        for channel in channels:
            print(f"Fetching from: {channel}")
            try:
                entity = await client.get_entity(channel)
                async for msg in client.iter_messages(entity, limit=100):
                    record = {
                        'channel': channel,
                        'message_id': msg.id,
                        'date': msg.date.isoformat(),
                        'sender_id': msg.sender_id,
                        'raw_text': msg.message,
                        'clean_text': preprocess_amharic(msg.message),
                        'media': None
                    }
                    # Download media if available
                    if msg.media:
                        media_path = f"data/raw/{channel}_{msg.id}"
                        await msg.download_media(media_path)
                        record['media'] = media_path
                    all_records.append(record)
            except Exception as e:
                print(f"Failed for {channel}: {e}")

        # Save to CSV
        df = pd.DataFrame(all_records)
        df.to_csv("data/clean/preprocessed_messages.csv", index=False, encoding="utf-8-sig")
        print("✅ Data ingestion and preprocessing complete.")
client = TelegramClient(session_name, api_id, api_hash)
# Define main function
async def main():
    await client.start(phone=phone)
    me = await client.get_me()
    print(f"✅ Logged in as: {me.first_name}")
    await client.disconnect()
# === Run it ===
if __name__ == "__main__":
    await main()


✅ Logged in as: Faya


In [8]:
import re
import pandas as pd

def extract_phone_numbers(text):
    if not text:
        return []
    pattern = r'\+?\d[\d\s\-]{7,}\d'
    return re.findall(pattern, text)

def extract_prices(text):
    if not text:
        return []
    pattern = r'\b\d{1,6}(\.\d{1,2})?\s?(birr|etb|₨)?\b'
    # Returns list of tuples, take first element of each
    matches = re.findall(pattern, text.lower())
    return [m[0] for m in matches]

async def fetch_messages(channel):
    print(f"Fetching from: {channel}")
    all_records = []
    try:
        entity = await client.get_entity(channel)
        async for msg in client.iter_messages(entity, limit=100):
            raw_text = msg.message or ""
            phones = extract_phone_numbers(raw_text)
            prices = extract_prices(raw_text)
            record = {
                'channel': channel,
                'message_id': msg.id,
                'date': msg.date.isoformat(),
                'text': raw_text,
                'phone_numbers': phones,
                'prices': prices,
            }
            all_records.append(record)
    except Exception as e:
        print(f"Failed for {channel}: {e}")
    return all_records

async def main():
    await client.start(phone=phone)
    all_data = []
    for channel in channels:
        records = await fetch_messages(channel)
        all_data.extend(records)

    # Save as CSV
    df = pd.DataFrame(all_data)
    df.to_csv("data/clean/messages_with_contacts_and_prices.csv", index=False, encoding="utf-8-sig")

    # Save as JSON
    with open("data/clean/messages_with_contacts_and_prices.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print("Saved CSV and JSON with text, phones, and prices.")
    await client.disconnect()

await main()


Fetching from: Leyueqa
Fetching from: nevacomputer
Fetching from: ZemenExpress
Fetching from: meneshayeofficial
Fetching from: ethio_brand_collection
Saved CSV and JSON with text, phones, and prices.


In [9]:
from telethon import TelegramClient, events
import os
import asyncio
import schedule
import time
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env

API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")

session_name = 'ethio_session'
phone = '+251954504193'

channels = [
    'Leyueqa',
    'nevacomputer',
    'ZemenExpress',
    'meneshayeofficial',
    'ethio_brand_collection'
]

os.makedirs('data/real_time/raw', exist_ok=True)
os.makedirs('data/clean', exist_ok=True)

client = TelegramClient(session_name, api_id, api_hash)

def preprocess_text(text):
    if not text:
        return ""
    return text.strip()

@client.on(events.NewMessage(chats=channels))
async def handler(event):
    msg = event.message
    data = {
        'channel': event.chat.username if event.chat else None,
        'message_id': msg.id,
        'date': msg.date.isoformat(),
        'sender_id': msg.sender_id,
        'text': preprocess_text(msg.message),
        'media': None
    }

    if msg.media:
        media_path = f"data/real_time/raw/{data['channel']}_{msg.id}"
        await msg.download_media(media_path)
        data['media'] = media_path

    df = pd.DataFrame([data])
    csv_path = "data/clean/messages_with_contacts_and_prices.csv"
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    else:
        df.to_csv(csv_path, mode='a', header=False, index=False, encoding="utf-8-sig")
    print(f"New message collected from {data['channel']}: {data['text'][:30]}...")

async def main():
    await client.start(phone=phone)
    print("Listening for new messages...")
    await client.run_until_disconnected()

def run_listener():
    print("Starting listener at scheduled time...")
    asyncio.run(main())  # Properly run async main in sync function

# Schedule to run every day at 08:00 AM
schedule.every().day.at("08:00").do(run_listener)

print("Scheduler started. Waiting for scheduled time...")

while True:
    schedule.run_pending()
    time.sleep(1)




Scheduler started. Waiting for scheduled time...


KeyboardInterrupt: 

In [10]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize

# Load CSV
df = pd.read_csv("data/clean/messages_with_contacts_and_prices.csv", encoding="utf-8-sig")

def preprocess_amharic(text):
    if not isinstance(text, str):
        return []
    # Remove URLs
    text = re.sub(r"http\S+", "", text)
    # Remove punctuation (you can customize for Amharic punctuation)
    text = re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", " ", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    # Lowercase (Amharic script doesn’t have case, but safe for mixed text)
    text = text.lower()
    
    # Optional: Normalize Amharic-specific chars (example: replace አ and ኣ to one form)
    # text = text.replace('አ', 'ኣ')  # example replacement, adjust as needed

    # Tokenize - using nltk (download punkt tokenizer before running)
    tokens = word_tokenize(text)
    
    return tokens

# Apply preprocessing and create new column
df['tokens'] = df['text'].apply(preprocess_amharic)

# Example: Save back to CSV or use df['tokens'] for further analysis
df.to_csv("data/clean/messages_preprocessed.csv", index=False, encoding="utf-8-sig")



In [11]:
import pandas as pd

# Load the preprocessed CSV with the tokens column
df = pd.read_csv("data/clean/messages_preprocessed.csv", encoding="utf-8-sig")

# The 'tokens' column contains lists stored as strings, so convert them back to lists
import ast

df['tokens'] = df['tokens'].apply(ast.literal_eval)

# Now print the first 10 rows of tokens
for i, tokens in enumerate(df['tokens'].head(10)):
    print(f"Record {i+1} tokens:", tokens)


Record 1 tokens: ['🔥🔥saachi', 'አንጀት', 'አርስ', 'ስቶቭ', 'ባለ', '2', 'ተች', 'ስቶቭ', '🔄an', 'environmentally', 'friendly', 'and', 'safe', 'plate', 'based', 'on', 'a', 'light', 'wave', '💯light', 'wave', 'technology', '💯safe', 'and', 'high', 'quality', '⛔️does', 'not', 'emit', 'smoke', 'and', 'flame', 'suitable', 'for', 'this', 'product', 'cooking', '🙂can', 'be', 'roasted', 'and', 'steamed', 'this', 'appliance', 'can', 'perform', 'a', 'variety', 'of', 'functions', 'including', 'a', 'multi', 'function', 'plate', 'has', 'a', 'heating', 'function', '🟢ግዜዎን', 'እና', 'ጉልበትዎን', 'የሚቆጥብ', 'ፈጣን', 'ስቶቭ', '🟢ባለ', '2', 'price', '9500', 'delivery', 'free', 'ክፍያዎን', 'ዕቃዉ', 'እጅዎ', 'ሲደርስ', 'በሞባይል', 'ባንኪንግ', 'መፈፀም', 'ይችላሉበተጨማሪ', 'ሁለት', 'ዕቃዎችን', 'ከ', '1000', 'ብር', 'በላይ', 'የሚተመኑ', '2', 'ዕቃዎችን', 'አንዴ', 'ሲገዙ', 'ስጦታ', 'እንልክለዎታለን', '0933334444', 'lemazez', 'z', '0944109295', 'lemaze', 'z', '0946242424', 'le', 'mazez']
Record 2 tokens: ['👉👉👉super', 'stretch', 'silicon', 'lids', '6', 'pack', '🔰የዕቃ', 'መሸፈኛ', 'ሲሊከን', 'ፕላስቲኮች'

In [12]:
df.sample(2)  # Random 10 rows (good for getting a feel of the dataset)



,channel,message_id,date,text,phone_numbers,prices,tokens
139,nevacomputer,8733,2025-03-20T12:43:37+00:00,NaN,[],[],[]
101,nevacomputer,8774,2025-06-11T13:56:52+00:00,LENOVO X1 YOGA\nProcessor: 11th‑Gen Intel Core...,"['+251912759900', '+251920153333']","['', '', '.8', '.7', '', '', '', '', '', '', '...","[lenovo, x1, yoga, processor, 11th‑gen, intel,..."


In [13]:
missing_tokens_count = df['tokens'].isna().sum()
print(f"🔎 Missing/Nan values in 'tokens' column: {missing_tokens_count}")
empty_tokens_count = df['tokens'].apply(lambda x: isinstance(x, list) and len(x) == 0).sum()
print(f"Empty token lists: {empty_tokens_count}")



🔎 Missing/Nan values in 'tokens' column: 0
Empty token lists: 190


In [14]:
df = df[df['tokens'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)].reset_index(drop=True)
df.to_csv("data/clean/structured_telegram_messages_cleaned.csv", index=False, encoding="utf-8-sig")
print(f"✅ Cleaned DataFrame: {len(df)} rows remaining")
 

✅ Cleaned DataFrame: 310 rows remaining


In [15]:
import pandas as pd
import re
import ast

# Load raw message CSV (you can also load from JSON and convert)
df = pd.read_csv("data/clean//structured_telegram_messages_cleaned.csv", encoding="utf-8-sig")

# If tokens are saved as strings, convert to actual list
if df['tokens'].dtype == object and isinstance(df['tokens'].iloc[0], str):
    df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x))

# Clean the text content
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[\"#$%&'()*+/<=>@[\]^_`{|}~፣።]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_text'] = df['text'].apply(clean_text)

# Extract tokens (simple whitespace-based for Amharic)
df['tokens'] = df['clean_text'].apply(lambda x: x.split())

# Extract flag if the message contains phone number or price
def contains_phone(text):
    return bool(re.search(r"\b09\d{8}\b", text))

def contains_price(text):
    return bool(re.search(r"(?:birr|ብር|\b\d{2,7}\b)", text, re.IGNORECASE))

df['contains_phone'] = df['text'].apply(contains_phone)
df['contains_price'] = df['text'].apply(contains_price)

# Optional: Create a 'content' column for easier reference
df['content'] = df['clean_text']

# Reorder and structure columns
metadata_cols = ['channel', 'message_id', 'date']
content_cols = ['text', 'clean_text', 'tokens', 'phone_numbers', 'prices']

df_structured = pd.DataFrame()
df_structured[metadata_cols] = df[metadata_cols]
df_structured['raw_text'] = df['text']
df_structured['clean_text'] = df['clean_text']
df_structured['tokens'] = df['tokens']
df_structured['contains_phone'] = df['contains_phone']
df_structured['contains_price'] = df['contains_price']

# Save to CSV
df_structured.to_csv("data/clean/structured_telegram_messages.csv", index=False, encoding="utf-8-sig")
print("✅ Structured data saved.")


✅ Structured data saved.


In [20]:
df.sample(3)

,channel,message_id,date,text,phone_numbers,prices,tokens,clean_text,contains_phone,contains_price,content
245,ethio_brand_collection,6081,2025-05-17T05:44:59+00:00,"Skechers Quantum flex \nsize 40,41,42,43\nPric...",['0920238243'],"['', '', '', '', '', '']","[Skechers, Quantum, flex, size, 40,41,42,43, P...","Skechers Quantum flex size 40,41,42,43 Price 3...",True,True,"Skechers Quantum flex size 40,41,42,43 Price 3..."
190,meneshayeofficial,951,2025-03-02T11:19:30+00:00,ለበለጠ መረጃ ከታች ያለውን ማስፈንጠሪያ ይጫኑ\n\nhttps://menes...,[],[],"[ለበለጠ, መረጃ, ከታች, ያለውን, ማስፈንጠሪያ, ይጫኑ]",ለበለጠ መረጃ ከታች ያለውን ማስፈንጠሪያ ይጫኑ,False,False,ለበለጠ መረጃ ከታች ያለውን ማስፈንጠሪያ ይጫኑ
236,ethio_brand_collection,6090,2025-05-28T09:05:44+00:00,"Skechers Out door taupe \nSize 40,41,42,43\nPr...",['0920238243'],"['', '', '', '', '', '']","[Skechers, Out, door, taupe, Size, 40,41,42,43...","Skechers Out door taupe Size 40,41,42,43 Price...",True,True,"Skechers Out door taupe Size 40,41,42,43 Price..."


In [17]:
empty_prices_count = df['prices'].apply(lambda x: isinstance(x, list) and len(x) == 0).sum()
print(f"Empty prices lists: {empty_prices_count}")

Empty prices lists: 0


# problem
price column has no exact price what to do
